## Power plant capacity

A large power company wants to plan its capacity expansion for the next 10 years. The forecasted demand in megawatts for year $t \in \{1, . . . , 10\}$ is denoted by $d_t$, and is provided in the following table:

year(t)|1|2|3|4|5|6|7|8|9|10
-------|-|-|-|-|-|-|-|-|-|--
$d_t$|10|22|31|35|39|44|50|57|62|70

The company can expand its energy production capacity at the beginning of each year. Two energy production alternatives are available: nuclear and wind.

* An investment of $c_n = 100$ is required per megawatt of nuclear power. Such an investment in the beginning of year t implies that the purchased capacity will be available during period t, t + 1, t + 2 and t + 3.
* Similarly, an investment of $c_w = 120$ is required per megawatt of wind power capacity. Such an investment in the beginning of year t implies that the purchased capacity will be available during period t, t + 1 and t + 2.

Governmental regulations require that at most 60 % of the companies power capacity be nuclear.
Assume that the initial capacity of the company is zero for nuclear and wind. 


Use mip to formulate and solve the problem of planning the least cost capacity expansion, while satisfying the forecasted demand.

### Parameters:
 * $T = \{1, . . . , 10\}$ is the set of periods (years) in the planning horizon.
 * $I = \{0, 1\}$ is the set of energy production alternatives (0=nuclear, 1=wind).
 * $d_t$ is the demand (in megawatts) of year $t \in T$.
 * $c_i$ is the investment cost to produce one megawatt of type $i \in I$ power.
 * $a_i$ is the number of years (after the year of purchase) the capacity for type i power is available.
 * $N^{max}$ is the maximum allowed ratio of nuclear power capacity with respect to total available capacity for each year t.

### Variables:
 * $x_{it} \ge 0$ is the amount of type i power capacity available in year t.
 * $y_{it} \ge 0$ is the amount of type i power capacity purchased at the beginning of year t.

### Model:
$$
\begin{array}{llll}
\min & \sum_{t \in T}{\sum_{i \in I}{c_i y_{it}}} \\
\textrm{s.t.} & \sum_{i \in I}{x_{it}} \ge d_t &\qquad \forall t \in T \\
              & x_{0t} \le N^{max} (\sum_{i \in I}{x_{it}}) &\qquad \forall t \in T \\
              & x_{it} = \sum_{l = max\{1, t - a_i\}}^{t}{y_{il}} &\qquad \forall i \in I, \forall t \in T \\
              & x_{it}, y_{it} \in \mathbb R_+ &\qquad \forall i \in I, \forall t \in T
\end{array}
$$

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [ ]:
# Setup your parameters
T = range(10)
I = [0, 1] # 0 = nuclear, 1 = wind

d = [10, 22, 31, 35, 39, 44, 50, 57, 62, 70]
c = [100, 120]
a = [3, 2]

max_N = 0.6

In [ ]:
import mip

m = mip.Model()

x = [[ m.add_var() for t in T ] for i in I]
y = [[ m.add_var() for t in T ] for i in I]

for t in T:
    m.add_constr(mip.xsum(x[i][t] for i in I) >= d[t])

for t in T:
    m.add_constr(x[0][t] <= max_N * mip.xsum(x[i][t] for i in I))

for t in T:
    for i in I:
        m.add_constr(x[i][t] == mip.xsum(y[i][l] for l in range(max(0, t - a[i]), t + 1)))

m.objective = mip.minimize(mip.xsum(c[i] * y[i][t] for i in I for t in T))

status = m.optimize()
print(status)
print(m.objective_value)

for t in T:
  print("[{}] Nuclear: {:0.2f} \t Wind: {:0.2f}".format(t, y[0][t].x, y[1][t].x))